In [ ]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [ ]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data


data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [ ]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

# Calculating $\theta_2$

i.e. $\Sigma$ as in Nevo. For the purposes of this assignment, we do not have population charactersistics $D_i$

For now I am only looking at the diagnoal elements in $\Sigma$

I plan to make theta a $k \times 1$ vector of variance elements

In [ ]:
def cal_mu(v, x, sigma):
        """Calculate individual-specific utility"""

        pi = theta2
        sigma = np.diag(theta_2)  # off-diagonals of sigma are zero

        # these are nmkts by nsiminds by nvars arrays 
        sigma_v = (np.matmul(sigma,v))    
    return mu

In [ ]:
    def _cal_s(self, delta, mu):
        """Calculate market share
        Calculates individual choice probability first, then take sum"""       nsiminds = self.nsiminds
        ind_choice_prob = self.ind_choice_prob 

        _BLP.cal_ind_choice_prob(delta, mu, ind_choice_prob)
        s = ind_choice_prob.sum(axis=1) / nsiminds

        return s


In [ ]:
def cal_delta(v, x, sigma):
        """Calculate mean utility via contraction mapping"""

        niter = 0

        error = 1e-13  # tight tolerance

        mu cal_mu(v, x, sigma)

        while True:
            s = self._cal_s(delta, mu)
            diff = ln_s_jt - np.log(s)

            if np.isnan(diff).sum():
                raise Exception('nan in diffs')

            delta += diff

            if (abs(diff).max() < error) and (abs(diff).mean() < 1e-3):
                break

            niter += 1

        print('contraction mapping finished in {} iterations'.format(niter))

return delta

# Calculating $\theta_1$

i.e. $(\alpha, \beta)$ as in Nevo